In [1]:
import os
from datetime import datetime
import finnhub
import psycopg2
import psycopg2.extras
from dotenv import load_dotenv, find_dotenv

In [2]:
load_dotenv(find_dotenv())

True

In [3]:
api_key2 = os.environ.get("FINNHUB_API_KEY")
host2 = os.environ.get("DB_HOST")
database2 = os.environ.get("DB_NAME")
user2 = os.environ.get("DB_USER")
password2 = os.environ.get("DB_PASSWORD")

In [4]:
today = datetime.today().strftime('%Y-%m-%d')
finnhub_client = finnhub.Client(api_key=api_key2)

In [5]:
connection = psycopg2.connect(host=host2, database=database2, user=user2, password=password2)
cursor = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)

In [6]:
# Get all index_ids
cursor.execute("""SELECT * from indices""")
rows = cursor.fetchall()
rows

[['^GSPC', 'S&P 500', 1],
 ['^NDX', 'Nasdaq 100', 2],
 ['^DJI', 'Dow Jones Industrial Average', 3],
 ['^SP500-50', 'S&P Communication Services Select Sector', 4],
 ['^SP500-25', 'S&P Consumer Discretionary Select Sector', 5],
 ['^SP500-30', 'S&P Consumer Staples Select Sector', 6],
 ['^GSPE', 'S&P Energy Select Sector', 7],
 ['^SP500-40', 'S&P Financial Select Sector', 8],
 ['^SP500-35', 'S&P Health Care Select Sector', 9],
 ['^SP500-20', 'S&P Industrial Select Sector', 10],
 ['^SP500-15', 'S&P Materials Select Sector', 11],
 ['^SP500-60', 'S&P Real Estate Select Sector', 12],
 ['^SP500-45', 'S&P Technology Select Sector', 13],
 ['^SP500-55', 'S&P Utilities Select Sector', 14],
 ['^MID', 'S&P Mid-Cap 400', 15],
 ['^SP600', 'S&P SmallCap 600', 16],
 ['^OEX', 'S&P 100', 17],
 ['^RUI', 'Russell 1000', 18],
 ['^RUT', 'Russell 2000', 19],
 ['^R25I', 'Russell 2500', 20],
 ['^RUA', 'Russell 3000', 21],
 ['^RUMIC', 'Russell Microcap', 22],
 ['^RMCC', 'Russell Midcap', 23],
 ['^RT200', 'Russell

In [7]:
for index in rows:
    print(index['symbol'])
    c_dict = finnhub_client.indices_const(symbol = {index['symbol']})
    c_list2 = c_dict.get('constituents')
    c_list = list(set(c_list2))
    for c in c_list:
        cursor.execute("""
                    SELECT * FROM tickers WHERE symbol = %s
                """, (c,))
        stock = cursor.fetchone()
        try:
            if stock:
                cursor.execute("""
                        INSERT INTO index_holdings (index_id, holding_id, dt)
                        VALUES (%s, %s, %s)
                    """, (index['id'], stock['id'], today))
        except Exception as e:
            print(e)
            pass

^GSPC
^NDX
^DJI
^SP500-50
^SP500-25
^SP500-30
^GSPE
^SP500-40
^SP500-35
^SP500-20
^SP500-15
^SP500-60
^SP500-45
^SP500-55
^MID
^SP600
^OEX
^RUI
^RUT
^R25I
^RUA
^RUMIC
^RMCC
^RT200
^NDUEEGF
^NR736471
^NDUEACWF
^NDDUEAFE
^M7CN
^NDDUEMU
^STOXX50E
^STOXX
^GDAXI
^AXJO
TX60.TS
^FTSE


In [8]:
connection.commit()